# Install Transformers Library

In [22]:
!nvidia-smi
seed=42

Wed Jul 21 04:40:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

     |████████████████████████████████| 2.5 MB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 895 kB 61.7 MB/s 


In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
# from transformers import AutoModel, BertTokenizerFast
from transformers import RobertaTokenizer, RobertaModel

# specify GPU
device = torch.device("cuda")

# Load Dataset

In [33]:
df = pd.read_json('/content/drive/MyDrive/sarcasm detection/Datasets/reddit/sarcasm_detection_shared_task_reddit_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sarcasm detection/Datasets/reddit/sarcasm_detection_shared_task_reddit_testing.jsonl',lines=True)

df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()

,label,context,response,id,labels
1795,SARCASM,"[Spoiler Cristiane Justino vs. Amanda Nunes, H...",she will probably beat him too. she can fight ...,reddit_1796,0
1796,NOT_SARCASM,[Treyarch makes the most unbalanced game ever....,"No, but then the game really glitch(ed) out fo...",reddit_1797,1
1797,SARCASM,"[First Official Image from ""Zombieland: Double...",> Zombieland ~~2:~~ Double Tap They changed it...,reddit_1798,0
1798,SARCASM,[Its time to ban /r/The_Donald: Calling out sp...,We're not *supporting racists* and prospective...,reddit_1799,0
1799,SARCASM,[59 Alabama ministers sign a letter saying Roy...,Is she old enough to have Facebook,reddit_1800,0


In [6]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
np.random.seed(0)
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
!pip install contractions
import demoji
import contractions

demoji.download_codes()
nltk.download('stopwords') 
# nltk.download('wordnet')

     |████████████████████████████████| 321 kB 8.4 MB/s 
     |████████████████████████████████| 283 kB 50.9 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85453 sha256=1d3dd0f49ea1e91ec4c433e28a2dd22caaa5c176f98127e55ba9750a71b6f826
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
... OK (Got response in 0.15 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    # text = re.sub(r"@",'', text)
    # text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    # text = re.sub(r"USER",'@USER', text)
    # text = re.sub(r"USER",'', text)
    # text = re.sub(r'<URL>','',text)

    # text = demoji.replace_with_desc(text)
    # text = re.sub(r':','',text)
    text = re.sub(r'#\w+','',text)
    text = contractions.fix(text)
    text = re.sub(r"[\([{})\]]", "", text)
    # text = re.sub(r'\.+','',text)
    return text


In [10]:
for i in range(5):
  print(df['response'][i])
  print(preprocess_text(df['response'][i]))
  # print(preprocess_text2(df['response'][i]))
  print()
  # print(df['context_resp'][i])
  # print(preprocess_text(df['context_resp'][i]))
  # print(preprocess_text2(df['context_resp'][i]))
  # print()

Yeah I mean there's only one gender anyways, women are objects
Yeah I mean there is only one gender anyways, women are objects

Sounds like you don't like science, you theist sheep
Sounds like you do not like science, you theist sheep

Ofc play them in try mode, Blizzard were so generous and they gave you 1 lane bot 1v1 with creeps to play your favourite heroes all day!
Ofc play them in try mode, Blizzard were so generous and they gave you 1 lane bot 1v1 with creeps to play your favourite heroes all day!

I don't understand, Reddit told me that Hillary got a negative convention bump and that Trump had nowhere to go but up.
I do not understand, Reddit told me that Hillary got a negative convention bump and that Trump had nowhere to go but up.

yeh, they're the reigning triple premiers, why bring up 2008?
yeh, they are the reigning triple premiers, why bring up 2008?



In [12]:
# df['response'].values[:50]

In [15]:
def fun(A,l):
  if len(A)<=l:
    return A
  else:
    # return  A[-l::1]
    return A[-1:-l-1:-1]
def fun1(A):
  return ''.join(A)

In [19]:
l=0

# df['tweets']= df['response'] +'reply-'+ df['context'].apply(fun,args=[l]).apply(fun1)
# df1['tweets']=df1['response'] +'reply-'+ df1['context'].apply(fun,args=[l]).apply(fun1)

# df['tweets']= 'reply-'+ df['response'] + ' context- ' + df['context'].apply(fun,args=[l]).apply(fun1)
# df1['tweets']= 'reply-'+ df1['response'] +  ' context- ' +  df1['context'].apply(fun,args=[l]).apply(fun1)


# df['tweets']= df['context'].apply(fun,args=[l]).apply(fun1) + df['response']
# df1['tweets']= df1['context'].apply(fun,args=[l]).apply(fun1) + df1['response']

df['tweets']= df['response'].apply(preprocess_text) 
df1['tweets']=df1['response'].apply(preprocess_text)
# df['tweets']=df['tweets']+'reply-'+ df['response'].apply(preprocess_text)
# df1['tweets']=df1['tweets']+'reply-'+ df1['response'].apply(preprocess_text)

In [20]:
# check class distribution
df['labels'].value_counts(normalize = True)

1    0.5
0    0.5
Name: labels, dtype: float64

In [21]:
df1.loc[1628,'tweets']
# df.loc[2628,'labels']

'Fuck you for being 13 minutes early. '

In [ ]:
df1.loc[1628,'response']
# df.loc[2629,'labels']

In [ ]:
df1.loc[1628,'context']

In [ ]:
# df["tweets"]=df['response']
# df1["tweets"]=df1['response']

In [ ]:
df.shape

In [ ]:
# check class distribution
df['labels'].value_counts(normalize = True)

# Split train dataset into train, validation and test sets

In [23]:

X_s = df['tweets'].values
y_s = df['labels'].values

Xt_s = df1['tweets'].values
yt_s = df1['labels'].values

train_text, temp_text, train_labels, temp_labels = X_s,Xt_s,y_s,yt_s

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=seed, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)
val_text, test_text, val_labels, test_labels=temp_text, temp_text, temp_labels, temp_labels

# X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2, random_state=42)


In [17]:
# pip install -U sentence-transformers

# Import BERT Model and BERT Tokenizer

In [24]:
from transformers import RobertaTokenizer, TFRobertaModel
bert = RobertaModel.from_pretrained('roberta-large')

# Load the BERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [26]:
# output
print(sent_id)

{'input_ids': [[0, 9226, 16, 10, 741, 2399, 1421, 35950, 2, 1, 1, 1], [0, 1694, 40, 2051, 12, 90, 4438, 10, 741, 2399, 1421, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


# Tokenization

In [27]:
# get length of all the messages in the train set
seq_len = [len(i.split()) for i in train_text]
sum(seq_len)/4400
# pd.Series(seq_len).hist(bins = 30)

14.477045454545454

In [35]:
# train_text[:50]

In [28]:
max_seq_len = 80

In [29]:
# tokenize and encode sequences in the training set

tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    padding=True,
    truncation=True,    
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    padding=True,
    truncation=True,
    return_token_type_ids=False
    
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    padding=True,
    truncation=True,
    return_token_type_ids=False
    
)

In [51]:
train_text[0].split()

['Yeah',
 'I',
 'mean',
 'there',
 'is',
 'only',
 'one',
 'gender',
 'anyways,',
 'women',
 'are',
 'objects']

In [52]:
df.loc[2339,'response']

'I actually had the opposite experience to the other people replying, I used atom for the longest time, took a few more extensions but I felt they worked better and just more how I\'d expect out the box - but then again I mixed things up and tried vscode again and it seems like the go support with the extension has improved a lot, I think either will be fine if you like the "visual studio" style it can\'t hurt to try VS, while atom seems to be more visually customizable, but either will be good and are worth trying out I\'m sure'

In [53]:
count=0
maxi=0
for i in range(4400):
  maxi=max(maxi,len(train_text[i].split()))
  if len(train_text[i].split())>60:
    count+=1
    print(i,len(train_text[i].split()),df['tweets'][i])
    # break
# count
maxi

2217 69 Also, it is possible the whoever posted this photo is the person in the photo and that something in this photo has relevance like she used to love the movie Back to The Future 2 and always wanted a hoverboard, or that shirt is hers or perhaps there is an exact photo of her sister Courtney standing in that exact spot wearing the same outfit but on a skateboard.
2312 66 If it was not super cold I would use a zip lock freezer bag with a hole cut out for the lens just to be extra careful but the real issue shooting outside in snow is your lens takes FOREVER to acclimate and expect it to be quite foggy for some time if you just take it from warm inside temperatures to cold and dry outside.
2339 105 I actually had the opposite experience to the other people replying, I used atom for the longest time, took a few more extensions but I felt they worked better and just more how I would expect out the box - but then again I mixed things up and tried vscode again and it seems like the go s

387

In [55]:
print(train_text[2462])
print(tokens_train.data['input_ids'][2462])

I am in the industry, when I freelance I use blender though most of the industry is maya and max, blender is quite good but keep in mind it is essentially 5 photoshop level complexity programs rolled into one package (though that is the case with pretty much all 3D animation software) on the 2d side opentoon which is the software studio Ghibli uses was just made free, definitely worth checking out
[0, 100, 524, 11, 5, 539, 6, 77, 38, 18150, 38, 304, 37086, 600, 144, 9, 5, 539, 16, 189, 102, 8, 19220, 6, 37086, 16, 1341, 205, 53, 489, 11, 1508, 24, 16, 5700, 195, 17190, 46491, 672, 13879, 1767, 6387, 88, 65, 3737, 36, 18401, 14, 16, 5, 403, 19, 1256, 203, 70, 155, 495, 19269, 2257, 43, 15, 5, 132, 417, 526, 5963, 1342, 3863, 61, 16, 5, 2257, 4535, 272, 25068, 3572, 2939, 21, 95, 2]


# Convert Integer Sequences to Tensors

In [56]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Create DataLoaders

In [57]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# Freeze BERT Parameters

In [58]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = True
# print(bert)

# Define Model Architecture

In [59]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(1024,256)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(256,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id,mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [60]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [61]:
# optimizer from hugging face transformers
from transformers import AdamW
# decayRate = 0.999
optimizer = AdamW(model.parameters(), lr = 2e-5)
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
# define the optimizer


# Find Class Weights

In [62]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[1. 1.]


In [63]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

In [65]:
from tqdm.auto import tqdm

# Fine-Tune BERT

In [66]:
# # function to train the model
# def train():
  
#   model.train()

#   total_loss, total_accuracy = 0, 0
  
#   # empty list to save model predictions
#   total_preds=[]
  
#   # iterate over batches
#   for step,batch in tqdm(enumerate(train_dataloader),total=4400//batch_size+1):
#   # for step in tqdm(range(len(train_dataloader))):
#     # batch=train_dataloader[step]
#     # progress update after every 50 batches.
#     # if step % 50 == 0 and not step == 0:
#     #   print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

#     # push the batch to gpu
#     batch = [r.to(device) for r in batch]
 
#     sent_id, mask, labels = batch

#     # clear previously calculated gradients 
#     model.zero_grad()        

#     # get model predictions for the current batch
#     preds = model(sent_id, mask)

#     # compute the loss between actual and predicted values
#     loss = cross_entropy(preds, labels)

#     # add on to the total loss
#     total_loss = total_loss + loss.item()

#     # backward pass to calculate the gradients
#     loss.backward()

#     # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
#     torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#     # update parameters
#     optimizer.step()
#     my_lr_scheduler.step()
#     # model predictions are stored on GPU. So, push it to CPU
#     preds=preds.detach().cpu().numpy()

#     # append the model predictions
#     total_preds.append(preds)

#   # compute the training loss of the epoch
#   avg_loss = total_loss / len(train_dataloader)
  
#   # predictions are in the form of (no. of batches, size of batch, no. of classes).
#   # reshape the predictions in form of (number of samples, no. of classes)
#   total_preds  = np.concatenate(total_preds, axis=0)

#   #returns the loss and predictions
#   return avg_loss, total_preds

# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [67]:
# # function for evaluating the model
# def evaluate():
  
#   print("\nEvaluating...")
  
#   # deactivate dropout layers
#   model.eval()

#   total_loss, total_accuracy = 0, 0
  
#   # empty list to save the model predictions
#   total_preds = []

#   # iterate over batches
#   for step,batch in tqdm(enumerate(val_dataloader),total=1800//batch_size+1):
#     # batch=val_dataloader[step]
#     # Progress update every 50 batches.
#     # if step % 50 == 0 and not step == 0:
      
#       # Calculate elapsed time in minutes.
#       # elapsed = format_time(time.time() - t0)
            
#       # Report progress.
#       # print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

#     # push the batch to gpu
#     batch = [t.to(device) for t in batch]

#     sent_id, mask, labels = batch

#     # deactivate autograd
#     with torch.no_grad():
      
#       # model predictions
#       preds = model(sent_id, mask)

#       # compute the validation loss between actual and predicted values
#       loss = cross_entropy(preds,labels)

#       total_loss = total_loss + loss.item()

#       preds = preds.detach().cpu().numpy()

#       total_preds.append(preds)

#   # compute the validation loss of the epoch
#   avg_loss = total_loss / len(val_dataloader) 

#   # reshape the predictions in form of (number of samples, no. of classes)
#   total_preds  = np.concatenate(total_preds, axis=0)

#   return avg_loss, total_preds

# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [68]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Start Model Training

In [69]:
# # set initial loss to infinite
# best_valid_loss = float('inf')

# # empty lists to store training and validation loss of each epoch
# train_losses=[]
# valid_losses=[]

# #for each epoch
# epochs=30
# for epoch in range(epochs):
     
#     print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
#     for param_group in optimizer.param_groups:
#         print('learning_rate- ',param_group['lr'])
#     #train model
#     train_loss,_ = train()
    
#     #evaluate model
#     valid_loss, _ = evaluate()
    
#     #save the best model
#     # if valid_loss < best_valid_loss:
#     best_valid_loss = valid_loss
#     torch.save(model.state_dict(), 'saved_weights.pt')


    
#     # append training and validation loss
#     train_losses.append(train_loss)
#     valid_losses.append(valid_loss)
#     pred=[]
#     with torch.no_grad():
#       for i in range(0,90):
#         preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
#         preds = preds.detach().cpu().numpy()
#         pred.append(preds)
#     pred1=np.array(pred)
#     pred2=pred1.reshape((1800,2))
#     p=pd.DataFrame(pred2)
#     p.to_csv('71_Rresponse.csv')
#     pred3 = np.argmax(pred2, axis = 1)
#     print(classification_report(test_y, pred3))
#     print(accuracy_score(test_y, pred3))

#     print(f'\nTraining Loss: {train_loss:.3f}')
#     print(f'Validation Loss: {valid_loss:.3f}')

# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
epochs=1
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss,_ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    # get predictions for test data
    !nvidia-smi
    # with torch.no_grad():
    #   preds = model(test_seq.to(device), test_mask.to(device))
    #   preds = preds.detach().cpu().numpy()
    # preds = np.argmax(preds, axis = 1)
    # print(classification_report(test_y, preds))
    # print(accuracy_score(test_y, preds))
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 1
  Batch    50  of    138.
  Batch   100  of    138.

Evaluating...
  Batch    50  of     57.
Mon Jul 19 12:46:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    32W /  70W |  12100MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-----

# Load Saved Model

In [70]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

# Get Predictions for Test Data

In [71]:
pred=[]
with torch.no_grad():
  for i in tqdm(range(0,90)):
    preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
    preds = preds.detach().cpu().numpy()
    pred.append(preds)
print(np.array(pred).shape)


(90, 20, 2)


In [72]:

pred1=np.array(pred)

In [73]:
pred2=pred1.reshape((1800,2))
pred2.shape

(1800, 2)

In [74]:
p=pd.DataFrame(pred2)
p.to_csv('71_Rresponse.csv')

In [75]:
pred3 = np.argmax(pred2, axis = 1)
pred3 

array([1, 1, 0, ..., 1, 0, 0])

In [76]:
print(classification_report(test_y, pred3))

              precision    recall  f1-score   support

           0       0.64      0.40      0.49       900
           1       0.56      0.77      0.65       900

    accuracy                           0.59      1800
   macro avg       0.60      0.59      0.57      1800
weighted avg       0.60      0.59      0.57      1800

